# 1. Transform + โหลด Dataset + สร้าง DataLoader

utils เป็นคำย่อมาจาก “utilities” แปลตรง ๆ คือ “เครื่องมือ/ฟังก์ชันเสริม”

โค้ดนี้เป็น ฟังก์ชันสำหรับเตรียมข้อมูลภาพ เพื่อให้โมเดล PyTorch สามารถนำไป train และ test

In [3]:
from torchvision import datasets, transforms
import torch

def get_dataloaders(train_dir, test_dir, batch_size=32):

# --- 1. Transform สำหรับ Training
    transform_train = transforms.Compose([
        transforms.Resize((224, 224)),                             # 224x224 px (ขนาดมาตรฐานของโมเดล ShuffleNet)
        transforms.RandomHorizontalFlip(),                         # พลิกภาพ
        transforms.RandomRotation(15),                             # หมุนภาพ 15 องศา
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # เปลี่ยนสีเล็กน้อย
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))     # # Normalize แต่ละช่อง R,G,B  (ค่า pixel ให้มี mean=0.5, std=0.5)
]) 
    
# --- 2. Transform สำหรับ Testing 
    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])

# --- 3. โหลด Dataset
    train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
    test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

# --- 4. สร้าง DataLoader
    train_loader = torch.utils.data.DataLoader(
        train_dataset, 
        batch_size=32,                                # กำหนดจำนวนภาพต่อ batch
        shuffle=True,                                 # สุ่มภาพทุก epoch เพื่อให้โมเดลเรียนรู้แบบไม่จำเจ
        
        num_workers=4)                                # จำนวน subprocess ที่ใช้โหลดภาพ (ช่วยให้โหลดเร็วขึ้น)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, 
        batch_size=32, 
        shuffle=False,                                # ไม่ต้อง shuffle ข้อมูล test
        num_workers=4)

# --- 5. คืนค่า DataLoader
    return train_loader, test_loader

# 2. สร้างโมเดล ShuffleNetV2 และคืนค่าโมเดล

Flow : Pretrained ShuffleNetV2 → Replace fc → Output layer = num_classes → Return model

In [4]:
def get_model(num_classes=8):
    model = models.shufflenet_v2_x1_0(pretrained=True)      # -- โหลดโมเดล ShuffleNetV2 ขนาด x1.0 พร้อม weights ที่ pretrained บน ImageNet
    num_ftrs = model.fc.in_features                         # -- ดึงจำนวน input features ของ fully connected layer เดิม
    model.fc = nn.Linear(num_ftrs, num_classes)             # -- สร้าง fully connected layer ใหม่ ให้ output เท่ากับจำนวน class ของเรา

    # -- คืนค่าโมเดลที่ปรับแล้ว
    return model

# 3. Train and Test ตัวโมเดล + พร้อมเก็บค่า loss กับ accuracy

In [5]:
import torch
from tqdm import tqdm


# ตอน ----- TRAIN
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0  # correct = จำนวนภาพที่ทายถูก, total = จำนวนภาพทั้งหมด

    for images, labels in tqdm(dataloader, desc="Training", unit="batch"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()      # เคลียร์ gradient เก่า
        outputs = model(images)    # forward pass
        loss = criterion(outputs, labels)  # คำนวณ loss
        loss.backward()            # backward pass (คำนวณ gradient)
        optimizer.step()           # อัปเดตน้ำหนักโมเดล

        # ---------- เก็บค่า loss ----------
        running_loss += loss.item()

        # ---------- เก็บค่า accuracy ----------
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # ---------- คำนวณค่าเฉลี่ย ----------
    train_loss = running_loss / len(dataloader)  # loss เฉลี่ยต่อ batch
    train_acc = 100 * correct / total             # accuracy (%) = correct / total * 100

    return train_loss, train_acc  # คืนค่า loss และ accuracy ของ epoch


# ตอน ------ TEST
def eval_one_epoch(model, dataloader, criterion, device):
    model.eval()
    loss_total = 0.0
    correct, total = 0, 0

    with torch.no_grad():  # ปิด gradient เพื่อประหยัด memory
        for images, labels in tqdm(dataloader, desc="Testing", unit="batch"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            # ---------- เก็บค่า loss ----------
            loss_total += loss.item()

            # ---------- เก็บค่า accuracy ----------
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # ---------- คำนวณค่าเฉลี่ย ----------
    test_loss = loss_total / len(dataloader)
    test_acc = 100 * correct / total

    return test_loss, test_acc  # คืนค่า loss และ accuracy ของ epoch

# 4. Run Train + สรุปผล + สร้างกราฟ

In [6]:
# --- 1.Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib
matplotlib.use('Agg')            # Agg backend → ป้องกัน error เมื่อรันบนเครื่องที่ไม่มีหน้าต่าง GUI
import matplotlib.pyplot as plt

from utils import get_dataloaders
from model_ShufflenetV2 import get_model
from training_utils import train_one_epoch, eval_one_epoch
from collections import defaultdict


# --- 2. ฟังก์ชันคำนวณ Per-class Accuracy
def per_class_accuracy(model, dataloader, classes, device):
    model.eval()
    class_correct = defaultdict(int)
    class_total = defaultdict(int)

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            for label, pred in zip(labels, predicted):
                class_total[label.item()] += 1
                if label.item() == pred.item():
                    class_correct[label.item()] += 1

    print("\nPer-class Accuracy:")
    for i, class_name in enumerate(classes):
        if class_total[i] > 0:
            acc = 100 * class_correct[i] / class_total[i]
            print(f"  {class_name:15s}: {acc:6.2f}% ({class_correct[i]}/{class_total[i]})")
        else:
            print(f"  {class_name:15s}: No samples")


# --- 3. ฟังก์ชัน main()
def main():
    # อุปกรณ์ที่ใช้ประมวลผล (GPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # path ของ train and test
    train_dir = "data/Training"
    test_dir = "data/Testing"
    classes = ["Green_Curry", "Khao_phat", "Khao_Soi", "Massaman_Curry",
           "Pad_Krapraw", "Pad_Thai", "SomTum", "Tom_yum"]

    train_loader, test_loader = get_dataloaders(train_dir, test_dir)

# --- 4. สร้างโมเดล Loss function + Optimizer (มาปรับจูนโมเดลตรงนี้นะ)
    model = get_model(num_classes=8).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    scheduler.step() # โมเดลจะ fine-tune น้ำหนักได้ดีขึ้น

# --- 5. Train + Test แต่ละ epoch
    epochs = 25                    # จำนวนการรันปายยย
    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []

    for epoch in range(epochs):
        print(f"\nEpoch [{epoch+1}/{epochs}]")

        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = eval_one_epoch(model, test_loader, criterion, device)

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        test_losses.append(test_loss)
        test_accuracies.append(test_acc)
                      # append ค่า loss & accuracy เพื่อสร้างกราฟทีหลัง

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%")

# --- 6. Per-class Accuracy หลังเทรนเสร็จ
    per_class_accuracy(model, test_loader, classes, device)

# --- 7. Plot results
    plt.figure(figsize=(12, 5))

    # Loss graph
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs+1), train_losses, label='Train Loss')
    plt.plot(range(1, epochs+1), test_losses, label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss per Epoch')
    plt.legend()

    # Accuracy graph
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs+1), train_accuracies, label='Train Accuracy')
    plt.plot(range(1, epochs+1), test_accuracies, label='Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Accuracy per Epoch')
    plt.legend()

    plt.tight_layout()
    plt.savefig("training_graph.png")
    print("Saved training_graph.png")


# --- 8. เรียกใช้งาน main
if __name__ == "__main__":
    main()

c:\Github\Project-CNNs-Group\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Github\Project-CNNs-Group\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Github\Project-CNNs-Group\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch sk

Using device: cuda:0

Epoch [1/25]


Testing: 100%|██████████| 26/26 [00:06<00:00,  3.84batch/s]


Train Loss: 1.3192 | Train Acc: 71.80%
Test Loss: 0.3494 | Test Acc: 94.48%

Epoch [2/25]


Testing: 100%|██████████| 26/26 [00:05<00:00,  4.59batch/s]


...

Train Loss: 0.0256 | Train Acc: 99.34%
Test Loss: 0.0598 | Test Acc: 98.28%

Epoch [24/25]


Testing: 100%|██████████| 26/26 [00:05<00:00,  4.44batch/s]


Train Loss: 0.0151 | Train Acc: 99.69%
Test Loss: 0.0531 | Test Acc: 98.53%

Epoch [25/25]


Testing: 100%|██████████| 26/26 [00:05<00:00,  4.37batch/s]


Train Loss: 0.0197 | Train Acc: 99.44%
Test Loss: 0.0714 | Test Acc: 97.91%

Per-class Accuracy:
  Green_Curry    :  98.00% (98/100)
  Khao_phat      :  99.00% (99/100)
  Khao_Soi       : 100.00% (110/110)
  Massaman_Curry :  96.00% (96/100)
  Pad_Krapraw    : 100.00% (100/100)
  Pad_Thai       :  90.00% (90/100)
  SomTum         : 100.00% (95/95)
  Tom_yum        : 100.00% (110/110)
Saved training_graph.png
